Libraries Used

In [34]:
import json as js
import cv2
import numpy as np
from os import listdir as ld
from os import path
from keras.models import Model #type: ignore
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Reshape #type: ignore
from keras.optimizers import Adam #type: ignore
from keras.utils import Sequence #type: ignore
from tqdm import tqdm

Loading Paths

In [3]:
with open("config_CNN.json",'r') as file:
    paths = js.load(file)

Defining model

In [35]:
def CNN_Extractor():
    inputs = Input(shape=(256, 256, 3))
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
    outputs = MaxPooling2D((2, 2))(x)
    model = Model(inputs, outputs)
    return model

Loading and croping bounding box for training

In [ ]:
training_path = paths["Train_resized"]
training_json = paths["Preprocessed_Train_json"]
cropped_train_path = paths['Cropped_Train_Images']

with open(training_json,'r') as file:
    Boxes = js.load(file)

for i in tqdm(ld(training_path)):
    images = path.join(training_path,i)
    img = cv2.imread(images)
    for j in Boxes:
        if i == j["img_id"]:
            x,y,w,h = [int(z) for z in j["bbox"]]
            cropped_img = img[y:y+h, x:x+w]
            cropped_img = cv2.resize(cropped_img, (256, 256), interpolation=cv2.INTER_LINEAR) 
            filename = j["label"] + "-" + i
            output_path = path.join(cropped_train_path, filename)
            cv2.imwrite(output_path, cropped_img)
            break

100%|██████████| 118287/118287 [10:44<00:00, 183.44it/s]


Loading and croping bounding box for validation

In [ ]:
val_path = paths["Validation_resized"]
val_json = paths["Preprocessed_Validation_json"]
cropped_val_path = paths['Cropped_Validation_Images']

with open(val_json,'r') as file:
    Boxes = js.load(file)

for i in tqdm(ld(val_path)):
    images = path.join(val_path,i)
    img = cv2.imread(images)
    for j in Boxes:
        if i == j["img_id"]:
            x,y,w,h = [int(z) for z in j["bbox"]]
            cropped_img = img[y:y+h, x:x+w]
            cropped_img = cv2.resize(cropped_img, (256, 256), interpolation=cv2.INTER_LINEAR) 
            filename = j["label"] + "-" + i
            output_path = path.join(cropped_val_path, filename)
            cv2.imwrite(output_path, cropped_img)
            break

100%|██████████| 5000/5000 [00:11<00:00, 437.51it/s]


Extracting features

In [ ]:
feature_extractor = CNN_Extractor()
image_dir = paths["Cropped_Train_Images"] 
features = []

for img_name in tqdm(ld(image_dir)):
    img_path = path.join(image_dir, img_name)
    img = cv2.imread(img_path)
    img = cv2.resize(img, (256, 256))
    img = img.astype('float32') / 255.0  
    img = np.expand_dims(img, axis=0)    
    feature = feature_extractor.predict(img, verbose=0)
    features.append(feature.squeeze())   

features_array = np.array(features)
np.save(paths["Saved_Training_Features"], features_array)

 15%|█▌        | 17969/117266 [13:22<1:15:38, 21.88it/s]